In [1]:

import numpy as np
from scipy.integrate import simps


CFF = 9.786e-3   # free–free absorption coefficient prefactor


# =========================================================
# Coulomb logarithm
# Te in K, nu in Hz
# =========================================================
def coulomb_logarithm(Te, nu):
    return 24.5 + np.log(Te) - np.log(nu)


# =========================================================
# Free–free absorption coefficient α_ff (cm^-1)
# ne, ni in cm^-3, Te in K, nu in Hz
# =========================================================
def alpha_ff(ne, Te, nu, Zi=1.0):
    lnL = coulomb_logarithm(Te, nu)
    ni = ne  # coronal plasma (quasi-neutrality)
    return (
        CFF
        * ne * ni * (Zi**2)* lnL
        / (nu**2 * Te**1.5)
        
    )


# =========================================================
# Optical depth τ_ff(z, ν)
# =========================================================
def optical_depth(z, alpha):
    """
    Optical depth computed along LOS.
    Assumes z increases toward observer (observer at z=0).
    """
    return np.array([
        simps(alpha[:i+1], z[:i+1])
        for i in range(len(z))
    ])


# =========================================================
# Brightness temperature TB(ν)
# TB = ∫ Te(z) exp[-τ(z)] α(z) dz
# =========================================================
def brightness_temperature(z, ne, Te, nu, Zi=1.0):
    """
    Brightness temperature for thermal free–free emission.
    """
    # Absorption coefficient
    alpha = alpha_ff(ne, Te, nu, Zi)

    # Optical depth
    tau = optical_depth(z, alpha)

    # Integrand of TB
    integrand = Te * np.exp(-tau) * alpha

    # Brightness temperature integral
    TB = simps(integrand, z)

    return TB


# LOS grid (cm), observer at z = 0
z = np.linspace(-1e10, 0.0, 2000)

# Coronal plasma profiles
ne = 1e8 * np.exp(z / 2e9)     # cm^-3
Te = 1.5e6 * np.ones_like(z)  # K

nu = 100e6  # 100 MHz

TB = brightness_temperature(z, ne, Te, nu)

print("Brightness Temperature @ %.1f MHz = %.2e K" % (nu/1e6, TB))



Brightness Temperature @ 100.0 MHz = 1.54e+05 K
